In [3]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install seaborn
!pip install sklearn 
!pip install spacy
!pip install spacy-wordnet
!python -m spacy download en_core_web_sm

!pip install gensim

!pip install SPARQLWrapper


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON

def toLocalId(input_id):
    return input_id.split("/")[len(input_id.split("/")) - 1]

def truncate(input_string,length):
    return (input_string[:length] + '..') if len(input_string) > length else input_string

# TODO: caching
sparql = SPARQLWrapper("https://innovonto-core.imp.fu-berlin.de/management/core/query")
sparql.setQuery("""
    PREFIX gi2mo: <http://purl.org/gi2mo/ns#>  
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX inov:<http://purl.org/innovonto/types/#>

    SELECT ?idea ?content WHERE {
      ?idea a gi2mo:Idea;
            gi2mo:content ?content;
            gi2mo:hasIdeaContest <https://innovonto-core.imp.fu-berlin.de/entities/ideaContests/i2m-bionic-radar>.
            
    }
    ORDER BY ASC(?idea)
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

#labels = list(map(lambda binding: toLocalId(binding["idea"]["value"]),results["results"]["bindings"]))
#labels = list(map(lambda binding: truncate(binding["content"]["value"],30),results["results"]["bindings"]))

labels = [(str(i) + " : " + str(truncate(binding["content"]["value"],60))) for i,binding in enumerate(results["results"]["bindings"])]
contents = list(map(lambda binding: binding["content"]["value"],results["results"]["bindings"]))

print(labels)

#for result in results["results"]["bindings"]:
#    print(result["content"]["value"])

['0 : Detect sickness in pets through uncharacteristic movement.', '1 : Use in sports like running to identify areas for improvement..', '2 : Could be used in cooking, to determine the extent to which w..', '3 : to find lost livestock ', '4 : Security identification of people, as well as emergency weat..', '5 : The technology can be used to monitor air craft.', '6 : The technology can be used to investigate how many active wi..', '7 : the device could be used in a detective manner that is to sa..', '8 : send it into a tornado or hurricane to help get pattern and ..', '9 : This could be used to detect cancers that may be in their ea..', '10 : It might be used as an alternative means of identity verific..', '11 : This could also be used to locate and follow individuals who..', '12 : It can be used for detecting movement in front of doors and ..', '13 : This technology could be used to extrapolate walking data fr..', '14 : Helping firefighters in ways to get into a house that is on ..', '

# Sentence Embeddings
1) Pretrained USE.v5 with a Transformer encoder

2) ELMO

3) TODO BERT via SpaCy

TODO https://medium.com/better-programming/the-beginners-guide-to-similarity-matching-using-spacy-782fc2922f7c


In [3]:
# TODO calculate embeddings and then store them somewhere (caching)
import tensorflow_hub as tf_hub
import seaborn as sns
import numpy as np

use_model = tf_hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
use_embeddings = use_model(contents)
print(use_embeddings.shape)

(581, 512)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
        corr,
        xticklabels=labels,
        yticklabels=labels,
        vmin=0,
        vmax=1,
        cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

messages = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    # Asking about age
    "How old are you?",
    "what is your age?",
] 

# TODO this does not work. see roam (April 7th, 2020)
#elmo = hub.KerasLayer('https://tfhub.dev/google/elmo/3',
#                            signature='default',
#                            output_key='elmo')
#embeddings = elmo(messages)["elmo"]
#print(embeddings.shape)
# Aufräumen
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
tokens_input = [["the", "cat", "is", "on", "the", "mat"],
                ["dogs", "are", "in", "the", "fog", ""]]
tokens_length = [6, 5]
embeddings = elmo(
    inputs={
        "tokens": tokens_input,
        "sequence_len": tokens_length
    },
    signature="tokens",
    as_dict=True)["elmo"]

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
embeddings = elmo(
    ["the cat is on the mat", "dogs are in the fog"],
    signature="default",
    as_dict=True)["elmo"]



With the default signature, the module takes untokenized sentences as input. The input tensor is a string tensor with shape [batch_size]. The module tokenizes each string by splitting on spaces.


input_tokens = tf.keras.layers.Input(shape=(1000,), name="input_tokens", dtype="string")
sequence_length = tf.constant([1000])

elmo_layer = hub.KerasLayer('https://tfhub.dev/google/elmo/3',
                            signature='tokens',
                            output_key='elmo',
                            trainable=True)

embedding_tokens = elmo_layer(inputs={'tokens': input_tokens,
 'sequence_len': sequence_length})
 
 
 # just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

# Finding Nearest Neighbors

See also:
https://scikit-learn.org/stable/modules/neighbors.html
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html



In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# TODO define the distance metric!
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors
# If metric is “precomputed”, X is assumed to be a distance matrix and must be square during fit

X = np.array(use_embeddings)
# The default metric is minkowski, and with p=2 is equivalent to the standard Euclidean metric
# Other possible metrics are: "cosine"
nbrs = NearestNeighbors(n_neighbors=6).fit(X)

# kneighbors(self[, X, n_neighbors, …]) -> Finds the K-neighbors of a point
#distances, indices = nbrs.kneighbors(np.array(use_embeddings[0]).reshape(1,-1))
distances, indices = nbrs.kneighbors(X)
#print(indices)
#print(distances)

#example = indices[1]

for i in range(0,10):
    example = indices[i]
    labeledExample = list(map(lambda i: contents[i],example))
    print("Nearest Neighbours of: " + labeledExample[0])
    for t in range(1,5):
        print("\t" + str(t) + ": " + labeledExample[t] + " d: " + '{:.3f}'.format(distances[i][t]))
    print()
    


Nearest Neighbours of: Detect sickness in pets through uncharacteristic movement.
	1: Identify movement patterns in feral animals acting strangely to see if they have rabies. d: 0.875
	2: Use the technology for pet cameras to track animal movement d: 0.952
	3: watch how animals move d: 0.974
	4: Detect criminals by their movement patterns such as their gait. d: 0.997

Nearest Neighbours of: Use in sports like running to identify areas for improvement in runners' strides, and improve on athletic performance
	1: Use it to analyze and improve the running form of track and field athletes. d: 0.750
	2: It can be used in athletic coaching or scouting to identify movement patterns of athletes.  d: 0.935
	3: Analyze athletes to see what part of them is either helping or hurting them in things such as running d: 0.936
	4: Athletes and coaches can use this to analyze movement in sports to improve performance. d: 0.943

Nearest Neighbours of: Could be used in cooking, to determine the extent to w

In [15]:
for i in range(0,10):
    print(contents[i])

Detect sickness in pets through uncharacteristic movement.
Use in sports like running to identify areas for improvement in runners' strides, and improve on athletic performance
Could be used in cooking, to determine the extent to which water is boiling.
to find lost livestock 
Security identification of people, as well as emergency weather and dangerous phenomenon.
The technology can be used to monitor air craft.
The technology can be used to investigate how many active wild animals there are in the field.
the device could be used in a detective manner that is to say that it could be useful to predicting behavioral patterns of say criminal offenders, students, to perform research perhaps by tracking the physical lives of athletes or top achieving businessman or for companies to isolate expected behaviors of their employees and how they anticipate employees to move during a work day to establish fair and true standards
send it into a tornado or hurricane to help get pattern and weather 

In [1]:
import spacy
#https://github.com/gancia-kiss/Bachelorarbeit
#https://spacy.io/universe/project/spacy-wordnet
# Load English tokenizer, tagger, parser, NER and word vectors
import en_core_web_sm
from spacy import displacy
#from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

nlp = en_core_web_sm.load()
#nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')

example_text = "This technology could be used to map a patient's recovery from severe disfigurement or burns and help show consistent patterns in rehabilitation. It can help pinpoint tiny variances in their movement that showcase limb and structural regrowth or rebrand and help a Doctor see how fast they are recovering."


# In[2]:
doc = nlp(example_text)

options = {"compact": True}
displacy.render(doc, style="dep",options=options)

# In[3]:
#doc2 = nlp(LONG_NEWS_ARTICLE)
#displacy.render(doc2, style="ent")
# Analyze syntax
#print(example_text)
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])



## Can we work with stopword lists here?


# wordnet object link spacy token with nltk wordnet interface by giving acces to
# synsets and lemmas 
#print(token._.wordnet.synsets())
#print(token._.wordnet.lemmas())

Noun phrases: ['This technology', "a patient's recovery", 'severe disfigurement', 'burns', 'consistent patterns', 'rehabilitation', 'It', 'tiny variances', 'their movement', 'limb', 'structural regrowth', 'rebrand', 'a Doctor', 'they']
Verbs: ['could', 'use', 'map', 'help', 'show', 'can', 'help', 'pinpoint', 'showcase', 'help', 'see', 'recover']


In [ ]:




# LEARNING SPACY

In [32]:
from spacy import displacy
import en_core_web_lg
nlp = en_core_web_lg.load()


In [42]:
import spacy
doc = nlp("This technology could be used to map a patient's recovery from severe disfigurement or burns and help show consistent patterns in rehabilitation. It can help pinpoint tiny variances in their movement that showcase limb and structural regrowth or rebrand and help a Doctor see how fast they are recovering.")
print(doc)

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)
    
for token in doc:
    print(token.text, token.pos_, token.dep_)
    #  print(spacy.explain(token.pos_))
    #print(spacy.explain(token.dep_))

#for word in doc:
#    if word.dep_ in ('xcomp', 'ccomp'):
#        print(''.join(w.text_with_ws for w in word.subtree))

#for ent in doc.ents:
#    print(ent.text, ent.start_char, ent.end_char, ent.label_)

#print('Dependency Labels:')
#options = {"compact": True}
#displacy.render(doc, style="ent",options=options)
dep_labels = []
for token in doc:
    while token.head != token:
        dep_labels.append(token.dep_)
        token = token.head
#print(dep_labels)

This technology could be used to map a patient's recovery from severe disfigurement or burns and help show consistent patterns in rehabilitation. It can help pinpoint tiny variances in their movement that showcase limb and structural regrowth or rebrand and help a Doctor see how fast they are recovering.
This technology technology nsubjpass used
a patient's recovery recovery dobj map
severe disfigurement disfigurement pobj from
burns burns conj disfigurement
consistent patterns patterns dobj show
rehabilitation rehabilitation pobj in
It It nsubj help
tiny variances variances dobj pinpoint
their movement movement pobj in
limb limb dobj showcase
structural regrowth regrowth conj limb
a Doctor Doctor nsubj see
they they nsubj recovering
This DET det
technology NOUN nsubjpass
could VERB aux
be AUX auxpass
used VERB ROOT
to PART aux
map VERB xcomp
a DET det
patient NOUN poss
's PART case
recovery NOUN dobj
from ADP prep
severe ADJ amod
disfigurement NOUN pobj
or CCONJ cc
burns NOUN conj
and

In [ ]:
import nltk
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk

example_text = contents[1]
tokenizer = nlp.Defaults.create_tokenizer(nlp)
tokens = tokenizer(example_text)

print(tokens)

dog = wn.synset('dog.n.01')

#port_synset = lesk(tokens, 'sports','n')
# NLTK has a million things that you can do with synsets: 
# see: https://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.wordnet

print(dog)
print(dog.definition())
#rint(dog.hypernym_paths())
print(dog.min_depth())


#for syn in wn.synsets('sports'):
#    print(syn, syn.definition())

# Text Summarization
1. Gensim

https://rare-technologies.com/text-summarization-with-gensim/

https://radimrehurek.com/gensim/summarization/summariser.html

Wow ist das schlecht.


2. Spacy

https://medium.com/better-programming/extractive-text-summarization-using-spacy-in-python-88ab96d1fd97
https://github.com/KevinPike/spacy-summary/blob/master/main.py



In [14]:
from gensim.summarization import keywords
from gensim.summarization import summarize

example_text = ' '.join(contents)
#print(example_text)
print('Summary:')
# This works only, if the input has more than one sentence.
print(summarize(example_text))
print('Keywords:')
print(keywords(example_text))

Summary:
Use in sports like running to identify areas for improvement in runners' strides, and improve on athletic performance Could be used in cooking, to determine the extent to which water is boiling.
the device could be used in a detective manner that is to say that it could be useful to predicting behavioral patterns of say criminal offenders, students, to perform research perhaps by tracking the physical lives of athletes or top achieving businessman or for companies to isolate expected behaviors of their employees and how they anticipate employees to move during a work day to establish fair and true standards send it into a tornado or hurricane to help get pattern and weather data This could be used to detect cancers that may be in their earliest forms.
For example, if we know what a normal liver looks like, if it were scanned and even the most microscopic change was present, this technology you detect it.
can be used to find people covered in an avalanche baby rocker with vibra

use
useful
uses
movement
likely
detect
detective
detecting
detection
detects
detected
detectives
people
help
helps
helpful
tracking
track
animals
animators
animation
identifying
identified
hand movements
monitoring
monitored
animal moves
secure
identify areas
certain
object
objects
helping firefighters
sports like running
locate
location
locations
locator
located
locating
better
person
persons
personal
personally
pattern
monitor air
weather
area
car
cars
security identification
monitors suspicious
home
homes
space
spaces
active
activity
activities
activate
actively
activating
sets
set
settings
setting
humans
human
motions
potential
potentially
large
analyzing
analyze
analyzes
time
times
timed
base
based
predicting behavioral patterns
criminal
criminals
technology
technologies
bodies
different
places
place
placed
placing
baby
babies
conditions
condition
device
devices
motion sensor
pedestrians moving
data
predict
predictive
predicted
rescue
rescued
measures
measure
measuring
individuals